In [1]:
import sys

import re

import glob

import os

import os.path

import cv2

import random

import numpy as np

import matplotlib

import matplotlib.pyplot as plt

import pandas as pd


In [2]:
def read_image(file_path):

    image   = cv2.imread(file_path, cv2.IMREAD_COLOR) 

   
    image = cv2.resize(image, (ROWS, COLS), interpolation = cv2.INTER_CUBIC)

    return image


In [55]:
def make_params(file_path):
    
    basename = os.path.basename(file_path)
    file_number = int(re.sub(r'\D','',basename))
    laser_strength =file_number //30
    
    #params=[]
    
    if 'Al_' in basename :
        df = pd.read_csv('Al.csv')
        df_Al= df[df["material"] == "Al"]
        Al_array = df_Cu.to_numpy()
        matparam=np.empty(0)
        matparam = np.delete(Al_array, 0, 1)
        
    params = np.empty(0)
    
    
    params = np.append(matparam, laser_strength)
    
    return params

    

In [56]:
def preprocess(images):

    count = len(images)

    data = np.ndarray((count, ROWS, COLS, CHANNELS), dtype = np.uint8)



    for i, image_file in enumerate(images):

        image = read_image(image_file)
    
        image_rgb = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        
        parameter = make_params(image_file)
        
        numar = np.empty([20, COLS, ROWS])
        
        for j in range(0, 20):
            
            numar[j] = np.full((COLS, ROWS), parameter[j])
            
       
        image_rgb_b = np.concatenate([image_rgb.transpose(2, 0, 1), numar],axis=0)
        
        #image_rgb = np.append(image_rgb.T, np.array([numar]), axis=0)
        
        data[i] = image_rgb_b.T
        
    

    data = data.astype('float32')

    data = data/255.0



    return data
            
        

In [5]:
#Deep Learning module

import tensorflow 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras import utils
from tensorflow.keras.utils import to_categorical


Model Initializer¶

In [6]:
def Material_model():



    model = Sequential()

    model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', kernel_initializer='he_normal', input_shape=(ROWS, COLS, CHANNELS)))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=(5, 5), activation='relu', kernel_initializer='he_normal'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(120, activation='relu', kernel_initializer='he_normal'))

    model.add(Dropout(0.2))


    model.add(Dense(84, kernel_initializer='he_normal', activation='relu'))

    model.add(Dense(2, activation='softmax'))

    model.summary()

    model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [7]:
ROWS = 28

COLS = 28

CHANNELS = 23


In [8]:
train_inputdirectory ='Dataset_Al/TrainData'


In [9]:
Da_train = glob.glob(train_inputdirectory + "/Damage/*.jpg")

Un_train = glob.glob(train_inputdirectory + "/NotDamage/*.jpg")


train_images = Da_train + Un_train

random.shuffle(train_images)

In [10]:
len(train_images)

1050

In [20]:
train_labels=[]

for i in train_images:
    if 'Damage.' in i:
        train_labels.append(0)
        
    if 'Notdamage.' in i:
        train_labels.append(1)

In [21]:
len(train_labels)

1050

In [28]:
test_inputdirectory='Dataset_Al/TestData'
Da_test = glob.glob(test_inputdirectory + "/Damage/*.jpg")

NotDa_test = glob.glob(test_inputdirectory + "/NotDamage/*.jpg")



test_images = Da_test + NotDa_test

random.shuffle(test_images)


test_labels=[]

for i in test_images:
    if 'Damage.' in i:
        test_labels.append(0)
        
    if 'Notdamage.' in i:
        test_labels.append(1)

In [29]:
len(test_images)

294

In [30]:
len(test_labels)

294

In [57]:
train_data = preprocess(train_images)
test_data=preprocess(test_images)

UnboundLocalError: local variable 'matparam' referenced before assignment